In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

21/10/03 17:31:13 WARN Utils: Your hostname, hti-server resolves to a loopback address: 127.0.1.1; using 192.168.0.105 instead (on interface eno1)
21/10/03 17:31:13 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/10/03 17:31:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
import numpy as np
import math

def lon2tile(lon, zoom=15):
    if lon is None:
        return None
    return math.floor((lon+180)/360*(2**zoom))

def lat2tile(lat, zoom=15):
    if lat is None:
        return None
    return math.floor((1-math.log(math.tan(lat*np.pi/180) + 1/math.cos(lat*np.pi/180))/np.pi)/2 * (2**zoom))

lat = 20.61708333
x = lat2tile(lat)
x

14465

In [3]:
from pyspark.sql.functions import udf
from pyspark.sql.types import LongType, StructType
lat2tile = udf(lat2tile, LongType())
lon2tile = udf(lon2tile, LongType())

In [46]:
df = spark.read.option("inferSchema", True).options(header=True).csv('Hanoi.csv')
columns_to_drop = ['a1', 'a2', 'b1', 'b2', 'c1', 'c2', 'd1', 'd2']
df = df.drop(*columns_to_drop)
df.show(2)

+-----------+-----------+---------------+
|   latitude|  longitude|population_2020|
+-----------+-----------+---------------+
|20.61708333|105.6773611|    9.926520985|
|20.62041667|105.6893056|    9.926520985|
+-----------+-----------+---------------+
only showing top 2 rows



In [47]:
df = df.withColumn("x", lon2tile("longitude"))
df = df.withColumn("y", lat2tile("latitude"))

df.show(2)

+-----------+-----------+---------------+-----+-----+
|   latitude|  longitude|population_2020|    x|    y|
+-----------+-----------+---------------+-----+-----+
|20.61708333|105.6773611|    9.926520985|26002|14465|
|20.62041667|105.6893056|    9.926520985|26004|14465|
+-----------+-----------+---------------+-----+-----+
only showing top 2 rows



In [48]:
from pyspark.sql.functions import col
df = df.groupBy("x", "y").sum("population_2020")

In [49]:
df.show(2)

+-----+-----+--------------------+
|    x|    y|sum(population_2020)|
+-----+-----+--------------------+
|26021|14445|    4735.38082074997|
|26002|14444|  1838.0823731899973|
+-----+-----+--------------------+
only showing top 2 rows



In [52]:
df.toPandas().to_csv('convert_population.csv', index=False)